<a href="https://colab.research.google.com/github/gowtham-dd/Data-Science-Basics/blob/main/EDA_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

gowthamdd_rollar_coaster_path = kagglehub.dataset_download('gowthamdd/rollar-coaster')
robikscube_introduction_to_exploratory_data_analysis_path = kagglehub.notebook_output_download('robikscube/introduction-to-exploratory-data-analysis')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [ ]:
df=pd.read_csv('/kaggle/input/rollar-coaster/coaster_db.csv')
df.head()

## Data Understanding

1. shape
2. describe
3. head
4. tail
5. dtypes
6. column names

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.info()

## Data Preparation

1. Drop irrelevent col and row
2. Find duplicates
3. Rename col
4. Freature Creation

In [ ]:
## Method 1

df=df[['coaster_name',
     #'Length', 'Speed',
     'Location', 'Status', 'Opening date',
     #     'Type',
     'Manufacturer',
     #'Height restriction', 'Model', 'Height',
      # 'Inversions', 'Lift/launch system', 'Cost', 'Trains', 'Park section',
       #'Duration', 'Capacity', 'G-force', 'Designer', 'Max vertical angle',
       #'Drop', 'Soft opening date', 'Fast Lane available', 'Replaced',
       #'Track layout', 'Fastrack available', 'Soft opening date.1',
       #'Closing date',
     'Opened',
     #'Replaced by', 'Website',
      # 'Flash Pass Available', 'Must transfer from wheelchair', 'Theme',
       #'Single rider line available', 'Restraint Style',
       #'Flash Pass available', 'Acceleration', 'Restraints', 'Name',
       'year_introduced', 'latitude', 'longitude',
     #'Type_Main',
      'opening_date_clean',
     #'speed1', 'speed2', 'speed1_value', 'speed1_unit',
       'speed_mph',
     'height_value',
     #'height_unit', 'height_ft',
       'Inversions_clean', 'Gforce_clean']].copy()

In [ ]:
## Method 2 Drop Command
##df.drop(['Column'],axis=1)

In [ ]:
df.shape
df.columns
df.dtypes

In [ ]:
## opening_date_clean convert data to respective dtype
df['opening_date_clean']=pd.to_datetime(df['opening_date_clean'])

In [ ]:
## Rename Col if needed
df.rename(columns={'coaster_name':'Coaster_Name',
                  'year_introduced':'Year_Introduced',
                  'latitude':'Latitude',
                  'longitude':'Longitude',
                  'opening_date_clean':'Opening_Date',
                  'speed_mph':'Speed_mph',
                  'height_value':'Height_Value',
                  'Inversions_clean':'Inversions'}).copy()

In [ ]:
df.isna().sum()## row wise

In [ ]:
## duplicate data
df.loc[df.duplicated()] ## whole duplicate

df.loc[df.duplicated(subset=['coaster_name'])]## Duplicate on particular column

In [ ]:
##Check Duplicate
df.query('coaster_name== "Crystal Beach Cyclone"')

In [ ]:
## Duplicates with same more columns
df.loc[df.duplicated(subset=['coaster_name','Location','Opening date'])]
# not dup
df.loc[~df.duplicated(subset=['coaster_name','Location','Opening date'])] ## gives non dup values


In [ ]:
#Remove dup
df=df.loc[~df.duplicated(subset=['coaster_name','Location','Opening date'])] \
.reset_index(drop=True).copy()

In [ ]:
## check after remove the data
df.loc[df.duplicated(subset=['coaster_name','Location','Opening date'])]


In [ ]:
df['year_introduced'].value_counts()

In [ ]:
ax=df['year_introduced'].value_counts().head(10).plot(kind='bar',title='Top 10 Coaster Introduced')
ax.set_xlabel('Top 10')
ax.set_ylabel('Year')

In [ ]:
ax=df['speed_mph'].plot(kind='hist',bins=20,title='Coaster Speed')
ax.set_xlabel('Speed')
ax.set_ylabel('Freq')


## Feature Relationship

1. Scatterplot
2. Heaatmap Correlation
3. Pairplot
4. Groupby comparison

In [ ]:
df.plot(kind='scatter',x='speed_mph',y='height_value')

In [ ]:
sns.scatterplot(data=df,x='speed_mph',y='height_value',hue='year_introduced')

In [ ]:
sns.pairplot(data=df,vars=['year_introduced','speed_mph','height_value','Inversions_clean','Gforce_clean'],hue='Manufacturer')
plt.show()

In [ ]:
df_cor=df[['year_introduced','speed_mph','height_value','Inversions_clean','Gforce_clean']].dropna().corr()

In [ ]:
sns.heatmap(df_cor,annot=True)

## QA

1. Location with fastest rollar coaster

In [ ]:
ax=df.query('Location !="Other"').groupby('Location')['speed_mph'].agg(['mean','count']).query('count>=10').sort_values('mean').plot(kind='barh',figsize=(12,5),title='Average Coast Speed By Location')
ax.set_xlabel('Speed')